Ce notebook résout un problème de planification de commandes en utilisant à la fois des méthodes classiques et une approche d'informatique quantique (l'algorithme QAOA).

# Formulation Mathématique Détaillée : de l'Optimisation Contrainte au QUBO

## 1. Le Problème d'Optimisation Binaire Quadratique avec Contraintes (BQP)

Le problème d'ordonnancement est initialement modélisé comme un **Problème d'Optimisation Binaire Quadratique avec Contraintes** (en anglais, *constrained Binary Quadratic Program* ou BQP).

Soit $x \in \{0, 1\}^n$ un vecteur de $n$ variables de décision binaires. Dans notre cas, $n = N \times T$, et les composantes de $x$ sont les variables $x_{i,t}$ où $i \in \{0, ..., N-1\}$ et $t \in \{0, ..., T-1\}$.

Le problème général s'écrit :

**Minimiser**
$$ f(x) = c^T x + x^T H x $$

**Sous les contraintes**
$$ A x = b \quad \text{(contraintes d'égalité)} $$
$$ G x \le h \quad \text{(contraintes d'inégalité)} $$

Où :
- $c \in \mathbb{R}^n$ est le vecteur des coûts linéaires. Dans notre notebook, $c$ contient les temps de départ $t$ pour chaque variable $x_{i,t}$.
- $H \in \mathbb{R}^{n \times n}$ est la matrice (généralement symétrique) des coûts quadratiques. Dans notre problème initial, $H$ est la matrice nulle car l'objectif est purement linéaire.
- $A, G$ sont des matrices et $b, h$ sont des vecteurs qui définissent les contraintes linéaires.

## 2. La Transformation en QUBO : Principe de Pénalisation

Les algorithmes quantiques comme VQE ou QAOA sont conçus pour trouver l'état fondamental d'un Hamiltonien, ce qui équivaut à résoudre un problème d'optimisation **sans contraintes**. Le formalisme standard pour cela est le **QUBO (Quadratic Unconstrained Binary Optimization)**.

L'objectif est de transformer notre BQP en un QUBO en éliminant les contraintes. La méthode standard est d'intégrer les contraintes dans la fonction objectif sous forme de **termes de pénalité**.

Un terme de pénalité est une fonction qui est nulle si la contrainte est satisfaite, et strictement positive (et idéalement grande) si elle est violée.

Soit une contrainte d'égalité $a_j^T x = b_j$ (la $j$-ème ligne de $Ax=b$). On peut la pénaliser en ajoutant le terme suivant à la fonction objectif :
$$ P_j (a_j^T x - b_j)^2 $$
De même, une contrainte d'inégalité $g_k^T x \le h_k$ est d'abord convertie en une contrainte d'égalité en introduisant une variable d'écart (slack variable) $s_k \ge 0$:
$$ g_k^T x + s_k = h_k $$
Puisque $s_k$ doit être positif, on peut l'exprimer en binaire, ce qui complexifie la formulation. Une approche plus directe pour les inégalités est de les pénaliser avec une fonction telle que :
$$ P_k' \cdot \max(0, g_k^T x - h_k)^2 $$
La fonction `QuadraticProgramToQubo` de Qiskit gère cette transformation de manière systématique.

## 3. Construction de la Fonction Coût QUBO

La nouvelle fonction objectif non contrainte $F(x)$ devient :
$$ F(x) = \underbrace{c^T x + x^T H x}_{\text{Objectif Original}} + \underbrace{\sum_{j} P_j (a_j^T x - b_j)^2}_{\text{Pénalités d'égalité}} + \underbrace{\sum_{k} P_k' (\text{pénalité d'inégalité})}_{\text{Pénalités d'inégalité}} $$

Les coefficients de pénalité $P_j, P_k'$ doivent être choisis suffisamment grands pour "forcer" la solution optimale de $F(x)$ à être une solution qui respecte les contraintes. Si $P$ est trop petit, la solution minimale pourrait violer une contrainte pour obtenir un meilleur score sur l'objectif original. S'il est trop grand, il peut créer un paysage énergétique trop "accidenté" (rugged) pour que l'optimiseur puisse s'y retrouver. Le choix de $P$ est un problème difficile en soi, mais Qiskit utilise des heuristiques pour le déterminer automatiquement.

En développant les termes quadratiques des pénalités, on peut regrouper tous les termes pour obtenir la forme QUBO finale. Par exemple, $(a_j^T x - b_j)^2 = (x^T a_j - b_j)(a_j^T x - b_j) = x^T (a_j a_j^T) x - 2b_j a_j^T x + b_j^2$.

La fonction coût finale $F(x)$ peut être réécrite sous la forme canonique QUBO :
$$ F(x) = x^T Q x + \text{constante} $$
Où $Q$ est une nouvelle matrice symétrique qui absorbe l'objectif original $H$ et tous les termes quadratiques et linéaires issus des pénalités. La constante est ignorée car elle ne change pas la position du minimum.

## 4. Lien avec le Modèle d'Ising et les Hamiltoniens

Le formalisme QUBO est mathématiquement équivalent au **modèle d'Ising**, qui est le langage naturel de la physique statistique et des ordinateurs quantiques. La transformation est simple : on passe des variables binaires $x_i \in \{0, 1\}$ à des variables de spin $z_i \in \{-1, 1\}$ via la relation $x_i = (1 - z_i) / 2$.

La fonction QUBO $x^T Q x$ se transforme alors en un Hamiltonien d'Ising :
$$ \mathcal{H} = \sum_{i,j} J_{ij} Z_i Z_j + \sum_i h_i Z_i $$
Où $Z_i$ est l'opérateur de Pauli-Z agissant sur le $i$-ème qubit.

**Minimiser la fonction coût QUBO est alors équivalent à trouver l'état quantique $|\psi\rangle$ qui minimise l'espérance de l'Hamiltonien $\langle\psi|\mathcal{H}|\psi\rangle$. Cet état est, par définition, l'état fondamental de $\mathcal{H}$.**

C'est précisément ce que les algorithmes variationnels comme **QAOA** ou **VQE (Variational Quantum Eigensolver)** sont conçus pour faire : préparer un état quantique paramétré et ajuster les paramètres de manière itérative pour trouver l'état d'énergie minimale (l'état fondamental), qui correspond à la solution de notre problème d'optimisation initial.

# 1)Préparer l'environnement de travail

In [1]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

# Imports avec gestion d'erreurs
QISKIT_ALGORITHMS = False
SAMPLER_TYPE = "none"
CPLEX_AVAILABLE = False
COBYLA_CLASSICAL_AVAILABLE = False

print("🔍 Détection des modules disponibles...")

# Test qiskit-algorithms
try:
    from qiskit_algorithms import QAOA
    from qiskit_algorithms.optimizers import COBYLA
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_aer import AerSimulator
    QISKIT_ALGORITHMS = True
    print("qiskit-algorithms détecté")
    
    # Test du type de sampler
    try:
        from qiskit_aer.primitives import Sampler
        SAMPLER_TYPE = "aer_primitives"
        print("Sampler Aer primitives détecté")
    except ImportError:
        try:
            from qiskit.primitives import Sampler
            SAMPLER_TYPE = "qiskit_primitives"
            print("Sampler Qiskit primitives détecté")
        except ImportError:
            try:
                from qiskit.utils import QuantumInstance
                SAMPLER_TYPE = "quantum_instance"
                print("QuantumInstance détecté (ancienne méthode)")
            except ImportError:
                SAMPLER_TYPE = "none"
                print("Aucun sampler trouvé")
                
except ImportError:
    print("qiskit-algorithms non disponible")

# Test CPLEX
try:
    from qiskit_optimization.algorithms import CplexOptimizer
    CPLEX_AVAILABLE = True
    print("CPLEX détecté")
except ImportError:
    print("CPLEX non disponible")

# Test COBYLA classique
try:
    from qiskit_optimization.algorithms import CobylaOptimizer
    COBYLA_CLASSICAL_AVAILABLE = True
    print("COBYLA classique détecté")
except ImportError:
    print("COBYLA classique non disponible")

print("=" * 50)

🔍 Détection des modules disponibles...
qiskit-algorithms détecté
Sampler Aer primitives détecté
CPLEX détecté
COBYLA classique détecté


# 2)Configurer le cas d'usage spécifique

In [ ]:
g = 5  # granularité en secondes
H = 15  
T = H // g  # nombre de créneaux discrets
N = 2  # nombre de commandes

# Durée des commandes (en secondes)
durations = [5, 10]  # (RÉDUIT À 2 PETITES COMMANDES)
# Nombre de créneaux occupés par chaque commande
n_slots = [int(np.ceil(d/g)) for d in durations]

print(f"Paramètres du problème:")
print(f"- Horizon total: {H}s")
print(f"- Granularité: {g}s")
print(f"- Nombre de créneaux: {T}")
print(f"- Durées des commandes: {durations}s")
print(f"- Créneaux occupés par commande: {n_slots}")
print("=" * 50)

Paramètres du problème:
- Horizon total: 15s
- Granularité: 5s
- Nombre de créneaux: 3
- Durées des commandes: [5, 10]s
- Créneaux occupés par commande: [1, 2]


# 3)Définir les inconnues que l'optimiseur doit trouver

In [3]:
qp = QuadraticProgram()

# Définir les variables binaires x_{i,t}
for i in range(N):
    for t in range(T):
        qp.binary_var(name=f"x{i}_{t}")

print(f"Variables créées: {len(qp.variables)} variables binaires")


Variables créées: 6 variables binaires


# 4)Donner un but à l'optimisation
### On définit ce que l'on cherche à optimiser. Ici, on veut minimiser la somme des temps de début. En donnant un poids t à chaque variable x{i,t}, on incite l'algorithme à choisir des t les plus petits possibles

In [4]:
objective_linear = {}
for i in range(N):
    for t in range(T):
        var = f"x{i}_{t}"
        objective_linear[var] = t

qp.minimize(linear=objective_linear)
print("Fonction objectif définie: minimiser la somme des temps de début")


Fonction objectif définie: minimiser la somme des temps de début


# 5) S'assurer que la solution est réaliste et valide.
###  On ajoute les règles du jeu que la solution finale doit respecter :Chaque commande doit démarrer exactement une fois.
### Pas de chevauchement : Un créneau de temps ne peut être utilisé que par une seule commande à la fois.
### Pas de débordement : Une commande ne peut pas commencer si tard qu'elle se terminerait après l'horizon de temps total.

In [5]:
for i in range(N):
    vars_i = [f"x{i}_{t}" for t in range(T)]
    qp.linear_constraint(
        linear={var: 1 for var in vars_i}, 
        sense='==', 
        rhs=1, 
        name=f"start_once_{i}"
    )

print(f"Contraintes 'une fois par commande': {N} contraintes ajoutées")

# 4b️⃣ Contrainte : pas de chevauchement
for t in range(T):
    overlap_vars = {}
    for i in range(N):
        for start_slot in range(max(0, t - n_slots[i] + 1), min(T, t + 1)):
            if start_slot + n_slots[i] > t:
                var = f"x{i}_{start_slot}"
                if var not in overlap_vars:
                    overlap_vars[var] = 0
                overlap_vars[var] += 1
    
    if overlap_vars:
        qp.linear_constraint(
            linear=overlap_vars, 
            sense='<=', 
            rhs=1, 
            name=f"no_overlap_{t}"
        )

print(f"Contraintes de non-chevauchement: {T} contraintes ajoutées")

# 4c️⃣ Contrainte : ne pas dépasser l'horizon
for i in range(N):
    for t in range(T):
        if t + n_slots[i] > T:
            var = f"x{i}_{t}"
            qp.linear_constraint(
                linear={var: 1}, 
                sense='<=', 
                rhs=0, 
                name=f"no_overflow_{i}_{t}"
            )

print("Contraintes de non-débordement ajoutées")
print(f"Total des contraintes: {len(qp.linear_constraints)}")
print("=" * 50)


Contraintes 'une fois par commande': 2 contraintes ajoutées
Contraintes de non-chevauchement: 3 contraintes ajoutées
Contraintes de non-débordement ajoutées
Total des contraintes: 6


In [6]:
def solve_by_enumeration(): #force brute
    from itertools import product
    
    print("🔍 Énumération de toutes les solutions possibles...")
    print(f"🔢 Paramètres: T={T}, N={N}, n_slots={n_slots}")
    
    best_value = float('inf')
    best_solution = None
    valid_solutions = []
    total_combinations = T ** N
    
    print(f"Test de {total_combinations} combinaisons...")
    
    # Générer toutes les combinaisons possibles
    for idx, combination in enumerate(product(range(T), repeat=N)):
        if idx % 100 == 0 and idx > 0:
            print(f"  ... testé {idx}/{total_combinations} combinaisons")
        
        if is_valid_schedule(combination):
            obj_value = sum(combination)
            valid_solutions.append((combination, obj_value))
            print(f"Solution valide trouvée: {combination} (valeur: {obj_value})")
            
            if obj_value < best_value:
                best_value = obj_value
                best_solution = combination
    
    print(f"{len(valid_solutions)} solution(s) valide(s) trouvée(s)")
    
    if len(valid_solutions) == 0:
        print(" Aucune solution valide trouvée - problème potentiellement infaisable")
        print("🔍 Vérification des paramètres:")
        print(f"  - Durées: {durations}")
        print(f"  - Créneaux nécessaires: {n_slots}")
        print(f"  - Horizon: {T} créneaux ({H}s)")
        print(f"  - Somme des durées: {sum(durations)}s vs horizon {H}s")
    
    class EnumerationResult:
        def __init__(self, solution, value):
            self.x = [0] * len(qp.variables)
            self.fval = value
            self.status = "OPTIMAL" if solution is not None else "INFEASIBLE"
            
            if solution is not None:
                for i, start_time in enumerate(solution):
                    var_name = f"x{i}_{start_time}"
                    for idx, var in enumerate(qp.variables):
                        if var.name == var_name:
                            self.x[idx] = 1
                            break
    
    return EnumerationResult(best_solution, best_value)









In [7]:
def is_valid_schedule(combination):
    """Vérifier si une combinaison est valide"""
    schedule = [[] for _ in range(T)]
    
    # Vérifier que chaque commande ne dépasse pas l'horizon
    for i, start_time in enumerate(combination):
        if start_time + n_slots[i] > T:
            return False
        
        # Marquer les créneaux occupés
        for slot in range(start_time, start_time + n_slots[i]):
            if slot < T:  # Sécurité supplémentaire
                schedule[slot].append(i)
    
    # Vérifier qu'il n'y a pas de chevauchement
    for slot_commands in schedule:
        if len(slot_commands) > 1:
            return False
    
    return True

In [8]:
def solve_with_classical_optimizer():
    if CPLEX_AVAILABLE:
        try:
            print("Tentative avec CPLEX...")
            optimizer = CplexOptimizer()
            result = optimizer.solve(qp)
            return result, "CPLEX"
        except Exception as e:
            print(f"CPLEX échec: {e}")
    
    if COBYLA_CLASSICAL_AVAILABLE:
        try:
            print("Tentative avec COBYLA classique...")
            optimizer = CobylaOptimizer()
            result = optimizer.solve(qp)
            return result, "COBYLA_CLASSICAL"
        except Exception as e:
            print(f"COBYLA classique échec: {e}")
    
    print("🔄 Utilisation de l'énumération...")
    return solve_by_enumeration(), "ENUMERATION"

In [9]:
def solve_with_qaoa():
    if not QISKIT_ALGORITHMS:
        print("QAOA non disponible - qiskit-algorithms manquant")
        return None, "QAOA_UNAVAILABLE"
    
    try:
        print("Conversion en QUBO pour QAOA...")
        converter = QuadraticProgramToQubo()
        qubo = converter.convert(qp)
        print(f"QUBO: {len(qubo.variables)} variables")
        
        # Using StatevectorSampler to bypass the Aer simulator for debugging
        print("Configuration QAOA avec StatevectorSampler...")
        from qiskit.primitives import StatevectorSampler
        sampler = StatevectorSampler()
        
        qaoa = QAOA(
            sampler=sampler,
            optimizer=COBYLA(maxiter=30),
            reps=1
        )
        
        optimizer = MinimumEigenOptimizer(qaoa)
        print("🔄 Exécution de QAOA... (peut prendre quelques secondes)")
        
        result = optimizer.solve(qubo)
        return result, "QAOA"
        
    except Exception as e:
        print(f"Erreur QAOA: {e}")
        import traceback
        traceback.print_exc()
        return None, "QAOA_ERROR"

In [10]:
def display_solution(result, method_name):
    """Afficher une solution"""
    print(f"\n### Résolution avec {method_name} ###")
    print(f"Statut: {result.status}")
    print(f"Valeur optimale: {result.fval}")
    
    if result.x is not None and result.status in ["OPTIMAL", "SUCCESS"]:
        print("\nPlanification:")
        command_names = ["Pizza", "Burger", "Soda"]
        solution_found = False
        
        for i in range(N):
            for t in range(T):
                var_name = f"x{i}_{t}"
                var_index = None
                for idx, var in enumerate(qp.variables):
                    if var.name == var_name:
                        var_index = idx
                        break
                
                if var_index is not None and result.x[var_index] > 0.5:
                    start_time = t * g
                    end_time = start_time + durations[i]
                    print(f"  {command_names[i]}: créneau {t+1} (temps {start_time}s-{end_time}s)")
                    solution_found = True
        
        if not solution_found:
            print("Aucune solution claire trouvée")
    
    return result.status in ["OPTIMAL", "SUCCESS"] if hasattr(result, 'status') else False

In [11]:
print("Résolution du problème...")
print("=" * 50)

# Résolution classique
classical_result, classical_method = solve_with_classical_optimizer()
classical_success = display_solution(classical_result, classical_method)

# Résolution quantique
print(f"\nÉtat des modules: QISKIT_ALGORITHMS={QISKIT_ALGORITHMS}, SAMPLER_TYPE={SAMPLER_TYPE}")

if QISKIT_ALGORITHMS and SAMPLER_TYPE != "none":
    print("\n" + "=" * 50)
    qaoa_result, qaoa_method = solve_with_qaoa()
    
    if qaoa_result is not None:
        qaoa_success = display_solution(qaoa_result, qaoa_method)
    else:
        print(f"{qaoa_method}: Pas de résultat")
else:
    print(f"\nQAOA sauté - Algorithmes: {'✅' if QISKIT_ALGORITHMS else '❌'}, Sampler: {SAMPLER_TYPE}")

Résolution du problème...
Tentative avec CPLEX...
CPLEX échec: "The 'CPLEX' library is required to use 'CplexOptimizer'. You can install it with 'pip install 'qiskit-optimization[cplex]''."
Tentative avec COBYLA classique...
COBYLA classique échec: 'Incompatible problem: The COBYLA optimizer supports only continuous variables'
🔄 Utilisation de l'énumération...
🔍 Énumération de toutes les solutions possibles...
🔢 Paramètres: T=3, N=2, n_slots=[1, 2]
Test de 9 combinaisons...
Solution valide trouvée: (0, 1) (valeur: 1)
Solution valide trouvée: (2, 0) (valeur: 2)
2 solution(s) valide(s) trouvée(s)

### Résolution avec ENUMERATION ###
Statut: OPTIMAL
Valeur optimale: 1

Planification:
  Pizza: créneau 1 (temps 0s-5s)
  Burger: créneau 2 (temps 5s-15s)

État des modules: QISKIT_ALGORITHMS=True, SAMPLER_TYPE=aer_primitives

Conversion en QUBO pour QAOA...
QUBO: 6 variables
Configuration QAOA avec StatevectorSampler...
🔄 Exécution de QAOA... (peut prendre quelques secondes)


c:\Users\sabri\Desktop\qubo\env\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\sabri\Desktop\qubo\env\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\sabri\Desktop\qubo\env\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])



### Résolution avec QAOA ###
Statut: OptimizationResultStatus.SUCCESS
Valeur optimale: 1.0


In [12]:
def verify_solution(result):
    """Vérifier une solution"""
    if result.x is None or result.status not in ["OPTIMAL", "SUCCESS"]:
        return False
    
    print("\n### Vérification ###")
    
    # Vérifier contraintes
    for i in range(N):
        starts = sum(1 for t in range(T) if result.x[qp.variables_index[f"x{i}_{t}"]] > 0.5)
        if starts != 1:
            print(f"Commande {i+1}: {starts} démarrages")
            return False
        print(f"Commande {i+1}: 1 démarrage")
    
    # Vérifier chevauchements
    schedule = [[] for _ in range(T)]
    for i in range(N):
        for t in range(T):
            if result.x[qp.variables_index[f"x{i}_{t}"]] > 0.5:
                for slot in range(t, min(T, t + n_slots[i])):
                    schedule[slot].append(i+1)
    
    overlap = False
    for t, commands in enumerate(schedule):
        if len(commands) > 1:
            print(f"Créneau {t+1}: chevauchement {commands}")
            overlap = True
        elif len(commands) == 1:
            print(f"Créneau {t+1}: commande {commands[0]}")
        else:
            print(f"Créneau {t+1}: libre")
    
    return not overlap

print("\n" + "=" * 50)
if classical_success:
    verify_solution(classical_result)

print("\n" + "=" * 50)
print("Résolution terminée!")
print(f"Environnement: Algorithms={QISKIT_ALGORITHMS}, Sampler={SAMPLER_TYPE}")



### Vérification ###
Commande 1: 1 démarrage
Commande 2: 1 démarrage
Créneau 1: commande 1
Créneau 2: commande 2
Créneau 3: commande 2

Résolution terminée!
Environnement: Algorithms=True, Sampler=aer_primitives


# Debugging Report: QUBO Scheduling Notebook

This report outlines the series of issues encountered and their resolutions while debugging the Qiskit-based QUBO notebook.

---

### Problem 1: `ImportError` for `QuantumInstance`

*   **The Issue:** The initial error was `cannot import name 'QuantumInstance' from 'qiskit.utils'`. This occurred because `QuantumInstance` is a deprecated feature from older Qiskit versions that has now been completely removed.

*   **The Solution:** We modernized the `solve_with_qaoa` function to align with the current Qiskit "Primitives" pattern. This involved removing the `QuantumInstance` logic and instead passing a `Sampler` object directly to the `QAOA` algorithm.

---

### Problem 2: `TypeError` in the Verification Cell

*   **The Issue:** A `TypeError: 'dict' object is not callable` appeared in the verification cell. The code was incorrectly using function-style parentheses `()` to access the `qp.variables_index` dictionary.

*   **The Solution:** We corrected the syntax to `qp.variables_index[...]`, using square brackets for proper dictionary key access. This allowed the verification logic to function as intended.

---

### Problem 3: `TypeError: Invalid circuits` (Primitive V1 vs. V2)

*   **The Issue:** We then faced a `TypeError: Invalid circuits, expected Sequence[QuantumCircuit]`. A `DeprecationWarning` in the logs (`Sampler has been deprecated... please use SamplerV2 instead`) revealed a version mismatch between the V2 `QAOA` algorithm and the V1 `Sampler` primitive being used.

*   **The Solution:** We updated the `solve_with_qaoa` function to explicitly import and instantiate `SamplerV2` from `qiskit_aer.primitives`, ensuring compatibility between the modern algorithm and the simulator primitive.

---

### Problem 4: `AerError: 'unknown instruction: QAOA'`

*   **The Issue:** A low-level error from the simulator, `AerError: 'unknown instruction: QAOA'`, indicated a bug within `qiskit-aer` itself, where it failed to correctly process the circuits generated by the `QAOA` algorithm.

*   **The Solution (Workaround):** To bypass this specific bug, we replaced the high-performance `AerSampler` with the `StatevectorSampler`. This is a simpler, more stable simulator from the core Qiskit library that, while slower, was able to correctly interpret the circuits.

---

### Problem 5: Slow Performance / Long Execution Time

*   **The Issue:** The `StatevectorSampler` resolved the crash but was too slow for the original 21-qubit problem due to its non-optimized, pure-Python implementation.

*   **The Solution:** To make the execution time practical, we reduced the problem size. By decreasing the number of commands and the time horizon, we created a minimal 6-qubit problem that could be solved very quickly, confirming the entire workflow was correct.